In [1]:
from selenium import webdriver  # 동적크롤링
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
import math

In [2]:
# data = pd.read_csv("C:/Users/User/big_project/preprocessed_seoul_kidscafe.csv")
data = pd.read_excel("C:/Users/User/big_project/merge_keyword_data.xlsx")

In [3]:
data.head()


,콘텐츠명,구명,새주소[도로명 주소],지번주소,좌표[X],좌표[Y],전화번호,주요시설,중분류,대분류,...,무료구분,운영일,휴관일,이용정원(기관),이용정원(개인),신청가능연령,대표이미지,시설소개,카테고리_키워드,전체_키워드
0,대모산 유아숲체험원,강남구,NaN,서울특별시 강남구 일원동 436,127.075627,37.479988,02-3423-6254,"모래놀이공간, 로프오르기, 그물오르기, 균형잡기, 나무미끄럼틀, 실외교구마당, 숲속...",숲체험,놀기,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"활동적인, 시원한, 쉼터, 놀이, 게임, 놀이시설, 실내놀이터, 놀이터, 아이와 함...","활동적인, 시원한, 쉼터, 놀이, 게임, 놀이시설, 실내놀이터, 놀이터, 아이와 함..."
1,일자산공원 유아숲체험원,강동구,서울특별시 강동구 천호대로206길 87,서울특별시 강동구 둔촌동 산 102-4,127.154279,37.535578,02-3425-6452,"새둥지놀이터, 통나무건너기, 나무오르기, 밧줄오르기, 숲교실, 흔들다리 건너기 등",숲체험,놀기,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"활동적인, 시원한, 쉼터, 놀이, 게임, 놀이시설, 실내놀이터, 놀이터, 아이와 함...","활동적인, 시원한, 쉼터, 놀이, 게임, 놀이시설, 실내놀이터, 놀이터, 아이와 함..."
2,개웅산 유아숲체험원,구로구,NaN,서울특별시 구로구 개봉동 산 53-3,126.847932,37.486117,02-860-3147,"유아쉼터, 모험놀이대, 흙·모래놀이터, 인디언집, 숲속교실, 초화원 등",숲체험,놀기,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"활동적인, 시원한, 쉼터, 놀이, 게임, 놀이시설, 실내놀이터, 놀이터, 아이와 함...","활동적인, 시원한, 쉼터, 놀이, 게임, 놀이시설, 실내놀이터, 놀이터, 아이와 함..."
3,인왕산 유아숲체험원,서대문구,서울특별시 서대문구 세검정로4길 152-46,서울특별시 서대문구 홍제동 산 1-1,126.952888,37.589444,02-330-1544,"종합밧줄놀이대, 나무오르기, 나무위의 집, 숲속교실, 실외교구마당, 세족장 등",숲체험,놀기,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"활동적인, 시원한, 쉼터, 놀이, 게임, 놀이시설, 실내놀이터, 놀이터, 아이와 함...","활동적인, 시원한, 쉼터, 놀이, 게임, 놀이시설, 실내놀이터, 놀이터, 아이와 함..."
4,능골산 유아숲체험원,구로구,NaN,서울특별시 구로구 고척동 산 8,126.848622,37.508645,02-860-3147,NaN,숲체험,놀기,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"활동적인, 시원한, 쉼터, 놀이, 게임, 놀이시설, 실내놀이터, 놀이터, 아이와 함...","활동적인, 시원한, 쉼터, 놀이, 게임, 놀이시설, 실내놀이터, 놀이터, 아이와 함..."


카카오맵 리뷰 크롤링 시작

In [4]:
def review_scraping(driver, query, review_list, review_cnt):
    reviews = [] # 각 업체별 리뷰 리스트
    review__count = 0
    url = "https://map.kakao.com/"
    driver.get(url)
    time.sleep(3)
    
    try:
        #검색창에 입력값
        # searchloc = '경기 성남시 분당구 황새울로335번길 옥된장'
        searchloc = query

        search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]') # 카카오맵 검색창
        search_area.send_keys(searchloc)  # 검색어 전달
        driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # 돋보기 클릭

        time.sleep(5)

        driver.find_element(By.XPATH, r'//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)  # 장소 탭
        time.sleep(2)
        # 장소에 대한 상세보기(URL)추출하여 새로운 창 들어가기
        link_element = driver.find_element(By.CSS_SELECTOR, 'a.moreview')
        link = link_element.get_attribute('href')
        driver.get(link)
        time.sleep(3)
        
        try:
            # <em> 태그에서 후기 수(review_count) 추출
            span_element = driver.find_element(By.XPATH, '//strong[@class="total_evaluation"]/span[@class="color_b"]')
            review_counts = int(span_element.text)
            review__count = review_counts
            print("후기 수 = ", review_counts)
            if review_counts > 100:
                review_count = 100
            else:
                review_count = review_counts
            print('사용 후기수 = ', review_count)
            num = ((review_counts-3)/5)
            num_final = math.ceil(num)
        except:
            review__count = 0
            print("no_review_count") 
            
        review_cnt.append(review__count)
        
        try:    
            # By.CSS_SELECTOR -> 더보기버튼 요소 찾기
            nxt_button_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > a'))
                    )
            print("nxt_button_find_success")  
  
            # 더보기 버튼 끝까지 누르기
            for i in range(num_final):
                nxt_button_element.find_element(By.XPATH, './span').click()
                time.sleep(0.4)
            print("nxt_button_success")
        except:
            print("nxt_button_find_fail")
            time.sleep(1)
        
        try:
            # By.CSS_SELECTOR -> ul 요소 찾기
            review_list_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > ul'))
                )
            print("ul_success")

            time.sleep(2)
            # 리뷰 추출
            for i in range(1, review_count+1): # 더보기 누르지 않은 상태로 최대 10개
                try:  # 사진 and 별점 없는 후기는 div 2번째에 텍스트 위치
                    review = review_list_element.find_element(By.XPATH, './li['+str(i)+']/div[3]/p/span').text
                    time.sleep(2)
                    reviews.append(review)      
                except: 
                    try:
                        review = review_list_element.find_element(By.XPATH, './li['+str(i)+']/div[4]/p/span').text
                        time.sleep(2)
                        reviews.append(review)
                    except:
                        try:
                            review = review_list_element.find_element(By.XPATH, './li['+str(i)+']/div[5]/p/span').text
                            time.sleep(2)
                            reviews.append(review)
                        except:
                            print(f"Review {i} not found in any of the expected locations")
                            reviews.append('텍스트없음')
        except:
            print("Review_find_fail")
            review_list.append('리뷰없음')
    except:
        review_list.append('검색불가')
        review_cnt.append(0)
        print('search fail!!')
        
    finally:
        #리뷰 저장
        if reviews:
            review_list.append(reviews)
                
        print(f'{query} - 리뷰 저장 완료')

In [30]:
# query = data['query']
# query = data['query'].iloc[:100]
# query = data['query'].iloc[100:200]
# query = data['query'].iloc[200:300]
# query = data['query'].iloc[300:400]
# query = data['query'].iloc[593:1000]
# query = data['query'].iloc[1000:1566]

query = data['query'].iloc[1000:1566]

In [31]:
query

1000          서울특별시 중구 명동길 14 CGV 명동
1001     서울특별시 노원구 동일로 1414 롯데시네마 노원
1002        서울특별시 강남구 강남대로 652 거암아트홀
1003       서울특별시 종로구 대학로8길 7 아르코예술극장
1004      서울특별시 종로구 동숭길 122 대학로극장 쿼드
                    ...             
1561     서울특별시 성북구 안암로 145 고려대학교 박물관
1562    서울특별시 양천구 목동동로 206-1 생명과학박물관
1563      서울특별시 중구 다산로36가길 25 박정희 가옥
1564             서울 종로구 돈화문로 71 떡박물관
1565    서울특별시 종로구 성균관로4길 45 짚풀생활사박물관
Name: query, Length: 566, dtype: object

In [11]:
from tqdm import tqdm

In [ ]:
driver = webdriver.Chrome()
kakao_rev = []
kakao_cnt = []
for i in tqdm(range(1000,1566), desc="Processing queries", ncols=100):
    tqdm.write(f'{i}번째 시작')
    q = query[i]
    review_scraping(driver, q, kakao_rev, kakao_cnt)

각 행에 알맞게 리뷰데이터 넣기

In [33]:
data_0_1000 = pd.read_csv('C:/Users/User/big_project/data_0_1000.csv')

In [34]:
# 첫 30개 행에 rev 리스트의 데이터 할당
for i in range(len(kakao_rev)):
    data_0_1000.at[i+1000, 'kakao_review'] = kakao_rev[i]
    data_0_1000.at[i+1000, 'kakao_review_cnt'] = kakao_cnt[i]